# Homework 1 - Professionalism & Reproducibility
# Name: James Joko (jjoko)

## License
The "CONSTANTS" cell, the `request_pageviews_per_article_mobile` function, and the `request_pageviews_per_article_desktop` function in this notebook contains code developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the Creative Commons [CC-BY license](https://creativecommons.org/licenses/by/4.0/). The following changes were made: REQUEST_HEADERS uses my UW email for the 'User-Agent', ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE has been changed to match the time range desired by the assignment and access types required by each question, the `request_pageviews_per_article_mobile` makes one request for each mobile type and sums the respon

For reproducibility, execute each cell from top to bottom. The file "thank_the_academy.AUG.2023.csv" must be located in the same folder as this notebook.

# Setup
Import the necessary modules we need for this homework. JSON for reading and storing API output, time for adding an artificial delay, urllib for URI encoding article titles, requests to make API requests, and pandas for reading the article dataset. We also load the article dataset into memory and define any constants we need for the API calls.

In [1]:
import json, time, urllib.parse, requests, pandas as pd

In [2]:
articles = pd.read_csv("thank_the_academy.AUG.2023.csv")
articles.head()

,name,url
0,Everything Everywhere All at Once,https://en.wikipedia.org/wiki/Everything_Every...
1,All Quiet on the Western Front (2022 film),https://en.wikipedia.org/wiki/All_Quiet_on_the...
2,The Whale (2022 film),https://en.wikipedia.org/wiki/The_Whale_(2022_...
3,Top Gun: Maverick,https://en.wikipedia.org/wiki/Top_Gun:_Maverick
4,Black Panther: Wakanda Forever,https://en.wikipedia.org/wiki/Black_Panther:_W...


In [3]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': 'jjoko@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015060100",   # start and end dates need to be set
    "end":         "2023093024"    
}

# Step 1: Data Acquisition
## 1. Monthly Mobile Access
To calculate monthly mobile access I make two requests for each article, one for each access type of mobile. I then sum the two outputs, remove the access field, and return the output as {article_title: \<time series data\>}. Once this process has been executed on all articles in the dataset, I save the output to a JSON file and verify the files structure.

In [ ]:
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015060100",   # start and end dates need to be set
    "end":         "2023093024"    
}

def request_pageviews_per_article_mobile(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'), safe='')
    request_template['article'] = article_title_encoded
    
    # Run a request for each mobile access type and store them
    mobile_access = ["mobile-app", "mobile-web"]
    json_response = []

    for access in mobile_access:
        request_template["access"] = access
        
        # now, create a request URL by combining the endpoint_url with the parameters for the request
        request_url = endpoint_url+endpoint_params.format(**request_template)

        # make the request
        try:
            # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
            # occurs during the request processing - throttling is always a good practice with a free
            # data source like Wikipedia - or other community sources
            if API_THROTTLE_WAIT > 0.0:
                time.sleep(API_THROTTLE_WAIT)
            response = requests.get(request_url, headers=headers)
            json_response.append(response.json())
        except Exception as e:
            print(e)
            return None
    # post-process the mobile json requests: sum the user counts, remove the access field
    for idx, item in enumerate(json_response[0]["items"]):
        del item["access"]
        item["views"] += json_response[1]["items"][idx]["views"]
    return {article_title: json_response[0]["items"]}

In [ ]:
monthly_mobile = {}
for article_title in articles["name"]:
    print(article_title)
    monthly_mobile.update(request_pageviews_per_article_mobile(article_title))

In [ ]:
with open('academy_monthly_mobile_201506-202309.json', 'w') as file:
    json.dump(monthly_mobile, file)

In [ ]:
with open('academy_monthly_mobile_201506-202309.json', 'r') as file:
    print(json.load(file)["Everything Everywhere All at Once"][:10])

## 2. Monthly Desktop Access
To calculate monthly desktop access I make a signle request for each article, specifying dekstop access. I then remove the access field and return the output as {article_title: \<time series data\>}. Once this process has been executed on all articles in the dataset, I save the output to a JSON file and verify the files structure.

In [ ]:
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015060100",   # start and end dates need to be set
    "end":         "2023093024"    
}

def request_pageviews_per_article_desktop(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'), safe='')
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = {article_title: response.json()["items"]}
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [ ]:
monthly_desktop = {}
for article_title in articles["name"]:
    monthly_desktop.update(request_pageviews_per_article_desktop(article_title))

In [ ]:
with open('academy_monthly_desktop_201506-202309.json', 'w') as file:
    json.dump(monthly_desktop, file)

In [ ]:
with open('academy_monthly_desktop_201506-202309.json', 'r') as file:
    print(json.load(file)["Everything Everywhere All at Once"][:5])

## 3. Monthly Cumulative
I already have the monthly time series of both mobile and desktop traffic per article. Firstly, I created a deepcopy of the monthly mobile traffic. Then, for every article, I start at the first month in the time series and add the desktop views to the mobile views for that month. After, I add the previous month's cumulative total to the views. Once this has been completed for each article, I write the resulting dictionary to storage.

In [ ]:
from copy import deepcopy
with open('academy_monthly_mobile_201506-202309.json', 'r') as file:
    monthly_mobile = json.load(file)
with open('academy_monthly_desktop_201506-202309.json', 'r') as file:
    monthly_desktop = json.load(file)

In [ ]:
monthly_cumulative = deepcopy(monthly_mobile)
for article_title in monthly_cumulative.keys():
    prev_views = 0
    for idx, month in enumerate(monthly_cumulative[article_title]):
        month["views"] += monthly_desktop[article_title][idx]["views"]
        month["views"] += prev_views
        prev_views = month["views"]

In [ ]:
with open('academy_monthly_cumulative_201506-202309.json', 'w') as file:
    json.dump(monthly_cumulative, file)

In [ ]:
with open('academy_monthly_cumulative_201506-202309.json', 'r') as file:
    print(json.load(file)["Everything Everywhere All at Once"][:5])

# Step 2: Analysis
For the analysis, I import matplotlib for time series visualization and datetime to properly format the x-axis

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

## Maximum Average and Minimum Average
To get the minimum and maximum average for each access type, I sum the views field of the time series and divide it by the length of the time series. If the average is greater than the max average seen so far or less than the min average seen so far, I save the article title and the average monthly views. 

In [ ]:
max_avg_desktop = (None, -1)
min_avg_desktop = (None, float("inf"))
for article_title in monthly_desktop.keys():
    avg_views = sum(month["views"] for month in monthly_desktop[article_title]) / len(monthly_desktop[article_title])
    if avg_views > max_avg_desktop[1]:
        max_avg_desktop = (article_title, avg_views)
    if avg_views < min_avg_desktop[1]:
        min_avg_desktop = (article_title, avg_views)
print(f"Maximum Average Desktop: {max_avg_desktop}")
print(f"Minimum Average Desktop: {min_avg_desktop}")

In [ ]:
max_avg_mobile = (None, -1)
min_avg_mobile = (None, float("inf"))
for article_title in monthly_mobile.keys():
    avg_views = sum(month["views"] for month in monthly_mobile[article_title]) / len(monthly_mobile[article_title])
    if avg_views > max_avg_mobile[1]:
        max_avg_mobile = (article_title, avg_views)
    if avg_views < min_avg_mobile[1]:
        min_avg_mobile = (article_title, avg_views)
print(f"Maximum Average Mobile: {max_avg_mobile}")
print(f"Minimum Average Mobile: {min_avg_mobile}")

To visualize the time series, I convert the timestamps to datetimes for x coordinates and views as the y coordinates for each of the min/max average per access type. I then specify axis labels and a correct y-axis scale to view all of the data. Without the log scale, pages with a low number of monthly views are overlapping lines at the bottom of the visual.

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
plt.xticks(rotation=-70)
plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_desktop[max_avg_desktop[0]]], [month["views"] for month in monthly_desktop[max_avg_desktop[0]]], label="Maximum Average Desktop: " + max_avg_desktop[0])
plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_desktop[min_avg_desktop[0]]], [month["views"] for month in monthly_desktop[min_avg_desktop[0]]], label="Minimum Average Desktop: " + min_avg_desktop[0])
plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_mobile[max_avg_mobile[0]]], [month["views"] for month in monthly_mobile[max_avg_mobile[0]]], label="Maximum Average Mobile: " + max_avg_mobile[0])
plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_mobile[min_avg_mobile[0]]], [month["views"] for month in monthly_mobile[min_avg_mobile[0]]], label="Minimum Average Mobile: " + min_avg_mobile[0])
plt.yscale("log")
plt.legend()
plt.ylabel("Log Number of Monthly Views")
plt.xlabel("Timestamp")
plt.title("Monthly Wikipedia Article Views over Time of Academy Award Winning Films with the Maximum and Minimum Average Monthly Views per Access Type")
plt.savefig("max_and_min_average.png")
plt.show()

## Top 10 Peak Page Views

To get the 10 articles with the highest peak page views per access type, I made a dictionary of the high peak page views per article per access type, sorted the dictionaries by descending value, and saved the article titles of the 10 highest peak page views.

In [ ]:
peak_views_mobile = {}
peak_views_desktop = {}
for article_title in monthly_mobile.keys():
    peak_views_mobile[article_title] = max(month["views"] for month in monthly_mobile[article_title])
    peak_views_desktop[article_title] = max(month["views"] for month in monthly_desktop[article_title])

In [ ]:
top_10_mobile = sorted(peak_views_mobile, key=peak_views_mobile.get, reverse=True)[:10]
top_10_desktop = sorted(peak_views_desktop, key=peak_views_desktop.get, reverse=True)[:10]

To visualize the time series, I convert the timestamps to datetimes for x coordinates and views as the y coordinates for each of the 10 articles with peak page views per access type. I then specify axis labels and a correct y-axis scale to view all of the data. Without the log scale, pages with a low number of monthly views are overlapping lines at the bottom of the visual.

In [ ]:
plt.rcParams["figure.figsize"] = (20,7)
plt.xticks(rotation=-70)
for article_title in top_10_desktop:
    plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_desktop[article_title]], [month["views"] for month in monthly_desktop[article_title]], label="Desktop: " + article_title)
for article_title in top_10_mobile:
    plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_mobile[article_title]], [month["views"] for month in monthly_mobile[article_title]], label="Mobile: " + article_title)
plt.legend(loc = 6, bbox_to_anchor=(1, 0.5))
plt.yscale("log")
plt.ylabel("Log Number of Monthly Views")
plt.xlabel("Timestamp")
plt.title("Monthly Wikipedia Article Views over Time of the Top 10 Academy Award Winning Films by Peak Page Views per Access Type")
plt.savefig("top10_peak_page_views.png", bbox_inches='tight')
plt.show()

## Fewest Months of Data

To get the 10 articles with the fewest months of data per access type, I made a dictionary of the number of months (the length of the time series returned by the API) per article per access type, sorted the dictionaries by ascending value, and saved the article titles of the 10 fewest months of data.

In [ ]:
num_months_mobile = {}
num_months_desktop = {}
for article_title in monthly_mobile.keys():
    num_months_mobile[article_title] = len(monthly_mobile[article_title])
    num_months_desktop[article_title] = len(monthly_desktop[article_title])
recent_10_mobile = sorted(num_months_mobile, key=num_months_mobile.get)[:10]
recent_10_desktop = sorted(num_months_desktop, key=num_months_desktop.get)[:10]

To visualize the time series, I convert the timestamps to datetimes for x coordinates and views as the y coordinates for each of the 10 articles with the fewest months of data per access type. I then specify axis labels and a correct y-axis scale to view all of the data. Without the log scale, pages with a low number of monthly views are overlapping lines at the bottom of the visual.

In [ ]:
plt.rcParams["figure.figsize"] = (20,7)
plt.xticks(rotation=-70)
for article_title in recent_10_desktop:
    plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_desktop[article_title]], [month["views"] for month in monthly_desktop[article_title]], label="Desktop: " + article_title)
for article_title in recent_10_mobile:
    plt.plot([datetime.strptime(month["timestamp"], "%Y%m%d%H") for month in monthly_mobile[article_title]], [month["views"] for month in monthly_mobile[article_title]], label="Mobile: " + article_title)
plt.legend(loc = 6, bbox_to_anchor=(1, 0.5))
plt.yscale("log")
plt.ylabel("Log Number of Monthly Views")
plt.xlabel("Timestamp")
plt.title("Monthly Wikipedia Article Views over Time of the Academy Award Winning Films with the Fewest Months of Data per Access Type")
plt.savefig("fewest_months_of_data.png", bbox_inches='tight')
plt.show()